In [8]:
import pandas as pd
import numpy as np
import math

In [13]:
df_made_save = pd.read_csv('saves.csv')
df_made_save = df_made_save.drop_duplicates(subset=['gID']).reset_index(drop=True)
df_made_save.head()

,gID,scoreDiff,inning,Current Outs,onFirst,onSecond,onThird,pID,name
0,ANA201004050,3,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
1,ANA201004230,2,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
2,ANA201004260,3,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
3,ANA201006220,3,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
4,ANA201006230,1,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes


In [14]:
df_pitch_subs = pd.read_csv('pitching_subs.csv')
df_pitch_subs.head()

/Users/dylanschneider/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (63,64,65,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,GAME_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,PITCH_SEQ_TX,AWAY_SCORE_CT,HOME_SCORE_CT,...,RUN3_FATE_ID,FATE_RUNS_CT,ASS6_FLD_CD,ASS7_FLD_CD,ASS8_FLD_CD,ASS9_FLD_CD,ASS10_FLD_CD,UNKNOWN_OUT_EXC_FL,UNCERTAIN_PLAY_EXC_FL,save_pit_id
0,ANA201004050,MIN,7,0,0,2,2,.CBFBX,3,4,...,0,0,0,0,0,0,0,F,F,fuenb001
1,ANA201004050,MIN,8,0,0,3,0,.BBBB,3,4,...,0,0,0,0,0,0,0,F,F,fuenb001
2,ANA201004050,MIN,9,0,0,1,2,.BCFS,3,6,...,0,0,0,0,0,0,0,F,F,fuenb001
3,ANA201004050,MIN,5,1,2,2,2,.CBFBX,3,4,...,0,0,0,0,0,0,0,F,F,fuenb001
4,ANA201004050,MIN,7,1,1,1,1,.CBX,3,4,...,0,0,0,0,0,0,0,F,F,fuenb001


In [15]:
def save_sit(row):
    if row['BAT_HOME_ID'] == 0 and row['HOME_SCORE_CT'] > row['AWAY_SCORE_CT']:
        return True
    elif row['BAT_HOME_ID'] == 1 and row['HOME_SCORE_CT'] < row['AWAY_SCORE_CT']:
        return True
    else:
        return False

df_pitch_subs['save_sit'] = df_pitch_subs.apply(save_sit, axis=1)

In [ ]:
df_pitch_subs[['GAME_ID', 'PIT_ID', 'INN_CT', 'BAT_HOME_ID', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'save_pit_id', 'save_sit']].head(100)

In [ ]:
df_pitch_subs = df_pitch_subs[df_pitch_subs['save_sit'] == True].reindex()

In [ ]:
df_pitch_subs[['GAME_ID', 'PIT_ID', 'INN_CT', 'BAT_HOME_ID', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'save_pit_id', 'save_sit']].head(10)

In [ ]:
df_next_pitcher = df_pitch_subs.shift(-1)
df_next_pitcher[['GAME_ID', 'PIT_ID', 'INN_CT', 'BAT_HOME_ID', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'save_pit_id', 'save_sit']].head()

In [ ]:
for t1, t2 in zip(df_pitch_subs.iterrows(), df_next_pitcher.iterrows()):
    cur_index = t1[0]
    cur_row = t1[1]
    next_row = t2[1]
    made_save = None
    
    if cur_row['PIT_ID'] == cur_row['save_pit_id']:
        made_save = True
        #df_pitch_subs['made_save'] = made_save
        continue
    
    if cur_row['GAME_ID'] == next_row['GAME_ID'] and cur_row['BAT_HOME_ID'] == next_row['BAT_HOME_ID']:
        # in same game, and same team.
        if cur_row['BAT_HOME_ID'] == 0:
            # home team
            blew_lead = next_row['HOME_SCORE_CT'] <= next_row['AWAY_SCORE_CT']
            if blew_lead:
                inn_diff = next_row['INN_CT'] - cur_row['INN_CT']
                init_score_diff = cur_row['HOME_SCORE_CT'] - cur_row['AWAY_SCORE_CT']
                
                if (init_score_diff <= 3 and inn_diff >= 1) or inn_diff >= 3 or score_diff + 2 + :
                    made_save = False
                
            
        elif cur_row['BAT_HOME_ID'] == 1
            # away team
            blew_lead = next_row['HOME_SCORE_CT'] >= next_row['AWAY_SCORE_CT']
            
    #df_pitch_subs['made_save'] = made_save
    
    
    break

In [5]:
for t1, t2 in zip(df_pitch_subs.iterrows(), df_next_pitcher.iterrows()):
    row = t1[1]
    print(row['BASE1_RUN_ID'])

NameError: name 'df_next_pitcher' is not defined